In [1]:
#Model: LSTM
#Word Embedding: Custom
#Dataset: 3
#based on: http://www.insightsbot.com/blog/1wAqZg/keras-lstm-example-sequence-binary-classification
#https://github.com/keras-team/keras/blob/master/examples/imdb_lstm.py
#https://github.com/keras-team/keras/blob/master/examples/imdb_bidirectional_lstm.py

import os

def GetTextFilePathsInDirectory(directory):
    files = []
    for file in os.listdir(directory):
        if file.endswith(".txt"):
            filePath = os.path.join(directory, file)
            files.append(filePath)
    return files

def GetLinesFromTextFile(filePath):
    with open(filePath,"r", encoding="utf-8") as f:
        lines = [line.strip() for line in f]
    return lines

In [2]:
positive_files = GetTextFilePathsInDirectory("./dataset3/positive")
negative_files = GetTextFilePathsInDirectory("./dataset3/negative")

reviews_positive = []
for i in range(0,1620):
    reviews_positive.extend(GetLinesFromTextFile(positive_files[i]))
    
reviews_negative = []
for i in range(0,1620):
    reviews_negative.extend(GetLinesFromTextFile(negative_files[i]))

In [4]:
import nltk
import re

REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])$&%=#•~@£©®→°€™›♥←×§″′Â█½à…“★”–●â►¢²¬░¶↑±¿▾¦║―¥▓▒¼⊕▼▪†■▀¨▄♫☆é♦¤▲è¾Ã∞∙↓、,│»♪╩╚³・╦╣╔╗▬❤ïØ¹≤‡√")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

def preprocess_reviews(reviews):
    default_stop_words = nltk.corpus.stopwords.words('german')
    stopwords = set(default_stop_words)
    
    reviews = [REPLACE_NO_SPACE.sub("", line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(" ", line) for line in reviews]
    reviews = [RemoveStopWords(line,stopwords) for line in reviews]
    
    return reviews

def RemoveStopWords(line, stopwords):
    words = []
    for word in line.split(" "):
        word = word.strip()
        if word not in stopwords and word != "" and word != "&":
            words.append(word)

    return " ".join(words)

 

In [5]:
reviews_positive = preprocess_reviews(reviews_positive)
reviews_negative = preprocess_reviews(reviews_negative)

In [6]:
import numpy as np
Reviews_Labeled = list(zip(reviews_positive, np.ones(len(reviews_positive))))
Reviews_Labeled.extend(list(zip(reviews_negative, np.zeros(len(reviews_negative)))))

In [7]:
from collections import Counter
import json
class Vocabulary:
    
    def __init__(self, vocabulary, wordFrequencyFilePath):
        self.vocabulary = vocabulary
        self.WORD_FREQUENCY_FILE_FULL_PATH = wordFrequencyFilePath
        self.input_word_index = {}
        self.reverse_input_word_index = {}
        
        self.MaxSentenceLength = None
        
    def PrepareVocabulary(self,reviews):
        self._prepare_Word_Frequency_Count_File(reviews)
        self._create_Vocab_Indexes()
        
        self.MaxSentenceLength = max([len(txt.split(" ")) for txt in reviews])
      
    def Get_Top_Words(self, number_words = None):
        if number_words == None:
            number_words = self.vocabulary
        
        chars = json.loads(open(self.WORD_FREQUENCY_FILE_FULL_PATH).read())
        counter = Counter(chars)
        most_popular_words = {key for key, _value in counter.most_common(number_words)}
        return most_popular_words
    
    def _prepare_Word_Frequency_Count_File(self,reviews):
        counter = Counter()    
        for s in reviews:
            counter.update(s.split(" "))
            
        with open(self.WORD_FREQUENCY_FILE_FULL_PATH, 'w') as output_file:
            output_file.write(json.dumps(counter))
                 
    def _create_Vocab_Indexes(self):
        INPUT_WORDS = self.Get_Top_Words(self.vocabulary)

        for i, word in enumerate(INPUT_WORDS):
            self.input_word_index[word] = i
        
        for word, i in self.input_word_index.items():
            self.reverse_input_word_index[i] = word
       
    def TransformSentencesToId(self, sentences):
        vectors = []
        for r in sentences:
            words = r.split(" ")
            vector = np.zeros(len(words))

            for t, word in enumerate(words):
                if word in self.input_word_index:
                    vector[t] = self.input_word_index[word]
                else:
                    pass
                
            vectors.append(vector)
            
        return vectors
    
    def ReverseTransformSentencesToId(self, sentences):
        vectors = []
        for r in sentences:
            words = r.split(" ")
            vector = np.zeros(len(words))

            for t, word in enumerate(words):
                if word in self.input_word_index:
                    vector[t] = self.input_word_index[word]
                else:
                    pass
                    #vector[t] = 2 #unk
            vectors.append(vector)
            
        return vectors

In [8]:
TOP_WORDS = 500
vocab = Vocabulary(TOP_WORDS,"analysislstmbidirectional.vocab")

reviews_text = [line[0] for line in Reviews_Labeled]
vocab.PrepareVocabulary(reviews_text)

In [9]:
reviews,labels=zip(*Reviews_Labeled)
reviews_int = vocab.TransformSentencesToId(reviews)

Reviews_Labeled_Int = list(zip(reviews_int,labels))

In [10]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(Reviews_Labeled_Int, test_size=0.2)

In [11]:
X_train, y_train = list(zip(*train))
X_test, y_test = list(zip(*test))

y_train = np.array(y_train)
y_test = np.array(y_test)

In [12]:
from keras.preprocessing import sequence 
max_review_length = 500 
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length) 
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length) 

Using TensorFlow backend.


In [13]:
from __future__ import print_function
import numpy as np

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional

max_features = 20000
# cut texts after this number of words
# (among top max_features most common words)
maxlen = 100
batch_size = 32

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')



58620 train sequences
14655 test sequences


In [14]:
print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('x_train shape:', X_train.shape)
print('x_test shape:', X_test.shape)
y_train = np.array(y_train)
y_test = np.array(y_test)

Pad sequences (samples x time)
x_train shape: (58620, 100)
x_test shape: (14655, 100)


In [15]:
model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(1, activation='sigmoid'))

Instructions for updating:
Colocations handled automatically by placer.


In [16]:
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

In [17]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 128)          2560000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               98816     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 2,658,945
Trainable params: 2,658,945
Non-trainable params: 0
_________________________________________________________________
None


In [18]:
print('Train...')
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=8,
validation_data=[X_test, y_test])


Train...
Instructions for updating:
Use tf.cast instead.
Train on 58620 samples, validate on 14655 samples
Epoch 1/8
58620/58620 [==============================] - 626s 11ms/step - loss: 0.5414 - acc: 0.7504 - val_loss: 0.5253 - val_acc: 0.7545
Epoch 2/8
58620/58620 [==============================] - 511s 9ms/step - loss: 0.5077 - acc: 0.7702 - val_loss: 0.5055 - val_acc: 0.7702
Epoch 3/8
58620/58620 [==============================] - 502s 9ms/step - loss: 0.4954 - acc: 0.7806 - val_loss: 0.5024 - val_acc: 0.7726
Epoch 4/8
58620/58620 [==============================] - 505s 9ms/step - loss: 0.4903 - acc: 0.7824 - val_loss: 0.5011 - val_acc: 0.7745
Epoch 5/8
58620/58620 [==============================] - 502s 9ms/step - loss: 0.4856 - acc: 0.7849 - val_loss: 0.4998 - val_acc: 0.7773
Epoch 6/8
58620/58620 [==============================] - 518s 9ms/step - loss: 0.4817 - acc: 0.7874 - val_loss: 0.4979 - val_acc: 0.7783
Epoch 7/8
58620/58620 [==============================] - 528s 9ms/step